# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [10]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [11]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [12]:
'''
提示：可參考以下的Stack Overflow: 
https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
'''

options = Options()
options.add_argument("--disable-notifications")

browser = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 0.01

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

import time
while True:
    print('網頁更新中...')
    time.sleep(5)
    browser.execute_script("window.scrollTo(0, 10000);")
    
    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端!")
        break
    last_height = new_height

/Users/b3620859/opt/anaconda3/envs/py36_env/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


網頁更新中...
網頁更新中...
到達頁面底端!


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [14]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [15]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 台股登上15400點新高！台積電收580元新天價　指數大漲249點收15463點

▲台股終場收在最高15463點。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
拜登獲國會認證，美國民主黨完全執政，美股道瓊、標普、那斯達克和費半全數收紅，四大指數同創新高，台股今日在台積電領軍下，指數開盤不到一分鐘即攻上15400點，終場收在最高15463點，上漲249點或1.64%，成交量3726.13億元，再度改寫歷史高點。
蘋概三王中，台積電（2330）美國存託憑證（ADR）攀高至121.43美元，續創歷史新高，大漲5.82%，帶動台積電今天股價延續強勁走勢，終場以最高580元作收、上漲2.91%，市值15.03兆元，同創新高且再創新天價。
鴻海（2317）終場以108元作收、上漲0.93%；股王大立光因昨天法說會釋出首季展望保守的大立光，早盤一度摜破 3000 元整數大關，但仍吸引部分低接買盤進場，跌幅逐漸收斂，向平盤附近靠攏，終場則是以3040元作收、上漲0.33%。其他大型權值也都上漲。
今日上市類股漲跌互見，表現較好為油電類股，上漲3.73％；表現較差為玻璃類股，下跌3.1％。
此外，IC設計廠聯發科股價也有不錯表現，一度達844元，市值1.34兆元，雙雙創下歷史新高。
國巨、華碩、矽力 - KY、緯穎等電子族群同樣吸引資金青睞，約有 2-4% 的漲幅。
----------------------------------------------------------------------

[1] 寒假打工要注意！　工讀生也要有這三大保障

▲勞保局提醒寒假打工須注意雇主是否有幫投保勞、就保。（圖片來源／rawpixel.com）
記者余弦妙／台北報導
剛滿18歲的黃同學他想利用今年寒假去賺一點額外零用錢，不過他不太確定自己未滿20歲是否也能加入勞保？對此，勞保局回應，根據勞保條例規定年滿15歲以上，65歲以下的勞工，其事業單位都必須幫其加入勞保。
勞保局解釋，只要是年滿15歲，65歲以下勞工受雇於5人以上的事業單位，就是參加勞工保險的被保險人，另外，就算事業單位的聘僱人數在5人以下，雖然不是勞保的強制投保單位，